# Data Consistency Check

This notebook checks if the number of neighborhoods in the processed data matches the input data for each city, indicator, and year.

In [2]:
import json
import pandas as pd
from pathlib import Path
import requests
from io import StringIO
from collections import defaultdict

In [17]:
import json
import pandas as pd
from pathlib import Path

# Configuración de rutas
BASE_DIR = Path.cwd().parent.parent
PROCESSED_FILES = {
    "barcelona": BASE_DIR / "data/processed/insert_ready_indicators_bcn.json",
    "madrid": BASE_DIR / "data/processed/insert_ready_indicators_madrid.json"
}

# Cargar los archivos
with open(PROCESSED_FILES['barcelona']) as f_bcn, open(PROCESSED_FILES['madrid']) as f_mad:
    data_bcn = json.load(f_bcn)
    data_mad = json.load(f_mad)

# Convertir a DataFrame
df_bcn = pd.DataFrame(data_bcn)
df_mad = pd.DataFrame(data_mad)

# Combinar ambos
df_all = pd.concat([df_bcn, df_mad], ignore_index=True)

# Agrupar y contar geo_id únicos por indicator_def_id, year, city_id
grouped = df_all.groupby(['indicator_def_id', 'year', 'city_id'])['geo_id'].nunique().reset_index()
grouped.rename(columns={'geo_id': 'unique_geo_ids'}, inplace=True)

# Mostrar resultados
print(grouped)

    indicator_def_id  year  city_id  unique_geo_ids
0                  1  2019        1              73
1                  1  2020        1              73
2                  1  2020        2             131
3                  1  2021        1              73
4                  1  2021        2             131
5                  1  2022        1              73
6                  1  2022        2             131
7                  1  2023        1              73
8                  1  2023        2             131
9                  1  2024        2             131
10                 2  2019        1              73
11                 2  2020        1              73
12                 2  2020        2             131
13                 2  2021        1              73
14                 2  2021        2             131
15                 2  2022        2             131
16                 2  2023        2             131
17                 2  2024        2             131
18          

In [21]:
import json
import pandas as pd
from pathlib import Path

# Configuración de rutas
BASE_DIR = Path.cwd().parent.parent
PROCESSED_FILES = {
    "barcelona": BASE_DIR / "data/processed/insert_ready_indicators_bcn.json",
    "madrid": BASE_DIR / "data/processed/insert_ready_indicators_madrid.json"
}

# Cargar los archivos
with open(PROCESSED_FILES['barcelona']) as f_bcn, open(PROCESSED_FILES['madrid']) as f_mad:
    data_bcn = json.load(f_bcn)
    data_mad = json.load(f_mad)

# Convertir a DataFrame
df_bcn = pd.DataFrame(data_bcn)
df_mad = pd.DataFrame(data_mad)

# === Barcelona ===
expected_bcn = df_bcn.groupby(['indicator_def_id', 'year'])['geo_id'].nunique().reset_index()
expected_bcn_total = expected_bcn.shape[0] * 73  # asumimos que deberían estar los 73 barrios en cada combinación
actual_bcn_total = len(df_bcn)

print("Barcelona:")
print(f"- Combinaciones únicas indicator/year: {expected_bcn.shape[0]}")
print(f"- Esperado (73 barrios por combinación): {expected_bcn_total}")
print(f"- Actual: {actual_bcn_total}")
print(f"- Completitud: {round(actual_bcn_total / expected_bcn_total * 100, 2)}%")
print()

# === Madrid ===
expected_mad = df_mad.groupby(['indicator_def_id', 'year'])['geo_id'].nunique().reset_index()
expected_mad_total = expected_mad.shape[0] * 131  # asumimos que deberían estar los 131 barrios en cada combinación
actual_mad_total = len(df_mad)

print("Madrid:")
print(f"- Combinaciones únicas indicator/year: {expected_mad.shape[0]}")
print(f"- Esperado (131 barrios por combinación): {expected_mad_total}")
print(f"- Actual: {actual_mad_total}")
print(f"- Completitud: {round(actual_mad_total / expected_mad_total * 100, 2)}%")

Barcelona:
- Combinaciones únicas indicator/year: 15
- Esperado (73 barrios por combinación): 1095
- Actual: 1095
- Completitud: 100.0%

Madrid:
- Combinaciones únicas indicator/year: 10
- Esperado (131 barrios por combinación): 1310
- Actual: 1310
- Completitud: 100.0%


In [22]:
# Contar registros en cada DataFrame
n_bcn = len(df_bcn)
n_mad = len(df_mad)
total = n_bcn + n_mad

print(f"Registros en Barcelona: {n_bcn}")
print(f"Registros en Madrid:    {n_mad}")
print(f"TOTAL para insertar:    {total}")

Registros en Barcelona: 1095
Registros en Madrid:    1310
TOTAL para insertar:    2405


In [23]:
n_indicators = df_all['indicator_def_id'].nunique()
n_neighborhoods = 73 + 131  # 204
n_districts = 10 + 21       # 31
n_cities = 2

expected_view_rows = n_indicators * (n_neighborhoods + n_districts + n_cities)

print(f"Indicadores únicos: {n_indicators}")
print(f"Filas esperadas en la vista: {expected_view_rows}")


Indicadores únicos: 4
Filas esperadas en la vista: 948
